# Art Institute of Chicago web scraper

In [ ]:
# Terms of use

# include AIC-User-Agent header:
headers = {"AIC-User-Agent": "artworks-semantic-search-application (annabozhenko2003@gmail.com)"}

In [5]:
from pathlib import Path 
import os

import json

from datasets import load_dataset

d:\artworks_search_engine\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Explore artpiece json structure

In [6]:
chicago_collection_dir = Path(r"D:\artworks_search_engine\data_retrieval\artic-api-data\json\artworks")


In [ ]:
# which entities are in the chicago  aside 
# from Prints and drawings and paintings
# 
# 

In [30]:
b

{'id': 100,
 'api_model': 'artworks',
 'api_link': 'https://api.artic.edu/api/v1/artworks/100',
 'is_boosted': False,
 'title': 'Simon Vouet',
 'alt_titles': None,
 'thumbnail': {'lqip': '',
  'width': 2215,
  'height': 3000,
  'alt_text': 'A work made of engraving in black on cream laid paper.'},
 'main_reference_number': '1887.716',
 'has_not_been_viewed_much': False,
 'boost_rank': None,
 'date_start': 1630,
 'date_end': 1636,
 'date_display': '1630/36',
 'date_qualifier_title': '',
 'date_qualifier_id': None,
 'artist_display': 'Robert van Voerst (Flemish, 1597-1636/37)\nafter Anthony van Dyck (Flemish, 1599-1641)',
 'place_of_origin': 'Flanders',
 'description': None,
 'short_description': None,
 'dimensions': 'Trimmed to platemark: 24.3 × 17.7 cm (9 5/8 × 7 in.)',
 'dimensions_detail': [{'depth': None,
   'width': 17,
   'height': 24,
   'diameter': None,
   'clarification': 'Trimmed to platemark'}],
 'medium_display': 'Engraving in black on cream laid paper',
 'inscriptions': No

In [69]:
important_fields = [
'title',
'classification_titles',
'subject_titles',
'technique_titles',
'material_titles',
'description',
'category_titles',
'artist_title',
'id',
'date_start',
'date_end',
'department_title',
'image_id']


In [70]:
chicago_collection_dir = Path(r"D:\artworks_search_engine\data_retrieval\artic-api-data\json\artworks")
artworks = {field: [] for field in important_fields}

for f in os.listdir(chicago_collection_dir):
    with open(Path(chicago_collection_dir, f)) as f:
        artwork = json.load(f)
        [artworks[field].append(artwork[field]) for field in important_fields]
        

In [14]:
from datasets import Dataset

In [71]:
ds = Dataset.from_dict(artworks)

In [72]:
list_ts = ['classification_titles',
 'subject_titles',
 'technique_titles',
 'material_titles',
 'category_titles']

ds = ds.map(lambda x: {f: ", ".join(x[f]) for f in list_ts})


Map: 100%|██████████| 134078/134078 [00:30<00:00, 4329.00 examples/s]


In [73]:
ds = ds.rename_columns({'classification_titles': 'classification',
                   'subject_titles': 'subjects',
                   'technique_titles': 'techniques',
                   'material_titles': 'materials',
                   'category_titles': 'categories',
                   'artist_title': 'artist',
                   })

In [74]:
full_info_fields = [
    'title',
    'classification',
 'subjects',
 'techniques',
 'materials',
 'description',
 'categories',
 'artist',
 'date_start',
 'date_end'
]

In [75]:

def safe_string(x):
    return '' if x is None else x

ds = ds.map(lambda x: {"full_info": ". ".join([f"{f}: {safe_string(x[f])}" 
                                                for f in full_info_fields])})


Map: 100%|██████████| 134078/134078 [00:35<00:00, 3796.41 examples/s]


In [76]:
ds = ds.map(lambda x: {"image_url": f"https://www.artic.edu/iiif/2/{x['image_id']}/full/843,/0/default.jpg"})

Map: 100%|██████████| 134078/134078 [00:36<00:00, 3690.63 examples/s]


In [78]:
ds.save_to_disk("./chicago_ds")

Saving the dataset (1/1 shards): 100%|██████████| 134078/134078 [00:00<00:00, 464933.00 examples/s]
